In [1]:
import pandas as pd
import oracledb
from tabulate import tabulate
import random
import re
import webbrowser
 
#SCRE
username = 'svcGOCUI'
password = 'DELL2023support#'
host = 'gocplorlvpr18.amer.dell.com'
port = '1521'
service_name = 'gooap_rw_oud_tls.prd.amer.dell.com'
 
#DAO GE4 SIT
# dbname="DAO_GE4"
# username = 'SIT_BROWSER'
# password = 'browse_s1t'
# host = 'udmnlorrrsi2a01.amer.dell.com'
# port = '1521'
# service_name = 'fdr4s.sit.amer.dell.com'
excel_name="Input_Order_Incoming_Data.xlsx"
def Retrive_VectorData():
    dsn = oracledb.makedsn(host, port, service_name=service_name)
    try :
        print("conn")
        connection = oracledb.connect(user=username, password=password, dsn=dsn)   
        print("Connected to Oracle Database")
        cursor = connection.cursor()
        # query = '''SELECT t.owner || '.' || t.table_name AS Table_Name,c.column_name as Column_Name FROM all_tables t 
        # JOIN all_tab_columns c ON t.table_name = c.table_name  ORDER BY t.table_name, c.column_name'''
 
        query = '''select a.region, to_char(a.processdate_utc,'yyyymmdd') processday,to_char(a.processdate_utc,'hh24') as processday_hour,
a.order_count,b.fiscalyear,b.fiscalquarter,b.fiscalmonth,b.fiscalweek,to_char(a.processdate_utc,'Day') as DayNum
  from work.tb_orderprocess_effectiveness a,work.tb_fiscalcalendar b
where to_char(a.processdate_utc,'yyyymmdd') between b.startdate and b.enddate
   and a.functionname = 'Order Incoming'
   and datatype = 'NEW'
   and vendor = 'ALL'
order by a.processdate_utc asc'''
 
        cursor.execute(query)
 
        rows = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
        data = pd.DataFrame(rows,columns=column_names)
        print(excel_name)
        data.to_excel(excel_name,index=False)
        table = tabulate(rows, headers=column_names, tablefmt="pretty")
        print(data)
        # data = pd.read_excel("DataFromDB.xlsx")

        # print(type(data))
        # print(data)
        return data
    except Exception as e:
       print(f"An error occurred: {e}")
    finally:
        if connection:
            connection.close()
            print("Connection closed")

In [2]:
data = Retrive_VectorData()

conn
Connected to Oracle Database
Input_Order_Incoming_Data.xlsx
      REGION PROCESSDAY PROCESSDAY_HOUR  ORDER_COUNT FISCALYEAR FISCALQUARTER  \
0        DAO   20230101              00          414       FY23            Q4   
1        APJ   20230101              00           27       FY23            Q4   
2       EMEA   20230101              00            5       FY23            Q4   
3        DAO   20230101              01          398       FY23            Q4   
4        APJ   20230101              01           29       FY23            Q4   
...      ...        ...             ...          ...        ...           ...   
40876    DAO   20240806              03          371       FY25            Q3   
40877    APJ   20240806              03          540       FY25            Q3   
40878   EMEA   20240806              05           88       FY25            Q3   
40879    APJ   20240806              05          516       FY25            Q3   
40880    DAO   20240806              05     

In [30]:
data.head()

,REGION,PROCESSDAY,PROCESSDAY_HOUR,ORDER_COUNT,FISCALYEAR,FISCALQUARTER,FISCALMONTH,FISCALWEEK,DAYNUM
0,APJ,20240207,09,508,FY25,Q1,M01,WK01,Wednesday
1,DAO,20240207,09,81,FY25,Q1,M01,WK01,Wednesday
2,EMEA,20240207,09,831,FY25,Q1,M01,WK01,Wednesday
3,EMEA,20240207,10,870,FY25,Q1,M01,WK01,Wednesday
4,APJ,20240207,10,377,FY25,Q1,M01,WK01,Wednesday


In [31]:
# # Uploading out put file into SQL DB
# import pandas as pd
# from sqlalchemy import create_engine
# from sqlalchemy.types import Integer, String, DateTime, Numeric
# import cx_Oracle
 
# #SCRE
# username = 'svcGOCUI'
# password = 'DELL2023support#'
# host = 'gocplorlvpr18.amer.dell.com'
# port = '1521'
# service_name = 'gooap_rw_oud_tls.prd.amer.dell.com'
# dsn = oracledb.makedsn(host, port, service_name=service_name)
 
# engine = create_engine(f'oracle+cx_oracle://{username}:{password}@{dsn}')
# dtype = {
#     'REGION': String(256), 
#     'FORECAST_DATE': DateTime,
#     'MODEL1': Numeric,
#     'MODEL2': Numeric,
#     'MODEL3': Numeric,
#     'MODEL4': Numeric,
#     'MODEL5': Numeric,
#     'ACTUALORDERUNIT': Numeric,
#     'CREATED_DATE': DateTime,
#     'UPDATED_DATE': DateTime,
# }
 
# try:
#     data = pd.read_csv(r"C:/Users/S_Munwar/Downloads/Final_Model_for_OIF/Future_forecasted_OIF_SQL.csv")
#     print(type(data))
#     data['FORECAST_DATE'] = pd.to_datetime(data['FORECAST_DATE'])
#     data['CREATED_DATE'] = pd.to_datetime(data['CREATED_DATE'])
#     data['UPDATED_DATE'] = pd.to_datetime(data['UPDATED_DATE'])
#     print(type(data))
#     data.to_sql('tb_orderprocess_effectiveness_forecast',schema ='work', con=engine, if_exists='append', index=False)
#     print("added successfully")
# except Exception as e:
#     print(f"An error occurred: {e}")
# finally:
#     engine.dispose()
#     print("Connection closed")

In [3]:
import pandas as pd

# Load data from CSV
# data = pd.read_csv("C:\\Users\\S_Munwar\\Downloads\\Input_Order_Incoming_Data.csv")
#data =pd.read_excel("C:\\Users\\S_Munwar\\Downloads\\Input_Order_Incoming_Data.xlsx")


# Convert PROCESSDAY to proper date format
data["PROCESSDAY"] = pd.to_datetime(data["PROCESSDAY"], format="%Y%m%d")

data["PROCESSDAY_HOUR"] = data["PROCESSDAY_HOUR"].astype(int)
# Create a new column combining PROCESSDAY and PROCESSDAY_HOUR
data["DateTime"] = data["PROCESSDAY"] + pd.to_timedelta(data["PROCESSDAY_HOUR"], unit="h")

# Drop the index column
#data = data.reset_index(drop=True)

In [4]:
#df= data.drop(columns=['PROCESSDAY', 'PROCESSDAY'])
df=data

In [5]:
# Separate dataframes for each region
emea_df = df[df["REGION"] == "EMEA"]
dao_df = df[df["REGION"] == "DAO"]
apj_df = df[df["REGION"] == "APJ"]

# Print the dataframes
print("EMEA DataFrame:")
print(emea_df)
print("\nDAO DataFrame:")
print(dao_df)
print("\nAPJ DataFrame:")
print(apj_df)

EMEA DataFrame:
      REGION PROCESSDAY  PROCESSDAY_HOUR  ORDER_COUNT FISCALYEAR  \
2       EMEA 2023-01-01                0            5       FY23   
5       EMEA 2023-01-01                1            8       FY23   
8       EMEA 2023-01-01                2            2       FY23   
15      EMEA 2023-01-01                5            6       FY23   
18      EMEA 2023-01-01                6            1       FY23   
...      ...        ...              ...          ...        ...   
40867   EMEA 2024-08-05               23           24       FY25   
40869   EMEA 2024-08-06                1            3       FY25   
40872   EMEA 2024-08-06                2           10       FY25   
40875   EMEA 2024-08-06                3           93       FY25   
40878   EMEA 2024-08-06                5           88       FY25   

      FISCALQUARTER FISCALMONTH FISCALWEEK     DAYNUM            DateTime  
2                Q4         M12       WK49  Sunday    2023-01-01 00:00:00  
5              

In [6]:
# Missing Values imputations for 3 regions
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Assuming df is your original DataFrame with all regions
emea_df = df[df["REGION"] == "EMEA"]
dao_df = df[df["REGION"] == "DAO"]
apj_df = df[df["REGION"] == "APJ"]

# Define a function for imputation
def perform_imputation(df, region_name):
    # Convert DateTime column to datetime format
    df["DateTime"] = pd.to_datetime(df["DateTime"])

    # Generate 1-hour intervals
    start_date = df["DateTime"].min()
    end_date = df["DateTime"].max()
    hour_intervals = pd.date_range(start=start_date, end=end_date, freq="H")

    # Create a new DataFrame with intervals
    interval_df = pd.DataFrame({"DateTime": hour_intervals})

    # Merge interval_df with original data
    merged_df = pd.merge(interval_df, df, on="DateTime", how="left")

    # Fill missing data based on available data
    merged_df["REGION"].fillna(method="ffill", inplace=True)
    merged_df["FISCALYEAR"].fillna(method="ffill", inplace=True)
    merged_df["FISCALQUARTER"].fillna(method="ffill", inplace=True)
    merged_df["FISCALMONTH"].fillna(method="ffill", inplace=True)
    merged_df["FISCALWEEK"].fillna(method="ffill", inplace=True)
    merged_df["DAYNUM"].fillna(method="ffill", inplace=True)

    # Perform imputation for ORDER_COUNT using DAYNUM
    daynum_order_count_avg = merged_df.groupby("DAYNUM")["ORDER_COUNT"].mean()
    merged_df["ORDER_COUNT"].fillna(merged_df["DAYNUM"].map(daynum_order_count_avg), inplace=True)

    # Fill remaining columns based on data & time columns
    merged_df["PROCESSDAY_HOUR"].fillna(merged_df["DateTime"].dt.hour, inplace=True)

    # Save the imputed DataFrame to an Excel file
    output_path = f"{region_name}_imputated.xlsx"
    merged_df.to_excel(output_path, index=False)

    # Print the resulting DataFrame
    print(f"Imputed data for {region_name} saved to {output_path}")

# Perform imputation for each region
for region_df, region_name in zip([emea_df, dao_df, apj_df], ["EMEA", "DAO", "APJ"]):
    perform_imputation(region_df, region_name)


C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_30052\2086939258.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DateTime"] = pd.to_datetime(df["DateTime"])
C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_30052\2086939258.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hour_intervals = pd.date_range(start=start_date, end=end_date, freq="H")
C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_30052\2086939258.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on wh

Imputed data for EMEA saved to EMEA_imputated.xlsx


C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_30052\2086939258.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DateTime"] = pd.to_datetime(df["DateTime"])
C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_30052\2086939258.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hour_intervals = pd.date_range(start=start_date, end=end_date, freq="H")
C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_30052\2086939258.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on wh

Imputed data for DAO saved to DAO_imputated.xlsx


C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_30052\2086939258.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DateTime"] = pd.to_datetime(df["DateTime"])
C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_30052\2086939258.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hour_intervals = pd.date_range(start=start_date, end=end_date, freq="H")
C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_30052\2086939258.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on wh

Imputed data for APJ saved to APJ_imputated.xlsx


In [36]:
emea_df.head()

,REGION,PROCESSDAY,PROCESSDAY_HOUR,ORDER_COUNT,FISCALYEAR,FISCALQUARTER,FISCALMONTH,FISCALWEEK,DAYNUM,DateTime
2,EMEA,2024-02-07,9,831,FY25,Q1,M01,WK01,Wednesday,2024-02-07 09:00:00
3,EMEA,2024-02-07,10,870,FY25,Q1,M01,WK01,Wednesday,2024-02-07 10:00:00
7,EMEA,2024-02-07,11,897,FY25,Q1,M01,WK01,Wednesday,2024-02-07 11:00:00
11,EMEA,2024-02-07,12,875,FY25,Q1,M01,WK01,Wednesday,2024-02-07 12:00:00
13,EMEA,2024-02-07,13,924,FY25,Q1,M01,WK01,Wednesday,2024-02-07 13:00:00


In [37]:
import pandas as pd

# Define a function to select specific columns from a DataFrame
def select_columns(df, columns):
    return df[columns].reset_index(drop=True)

# Define the columns to select
selected_columns = ["DateTime", "ORDER_COUNT"]

# Assuming emea_df, dao_df, and apj_df are your DataFrames for the EMEA, DAO, and APJ regions respectively
# Apply the function to each DataFrame
data_emea = select_columns(emea_df, selected_columns)
data_dao = select_columns(dao_df, selected_columns)
data_apj = select_columns(apj_df, selected_columns)

# Now you have the selected columns for each region
print("Selected columns for EMEA:")
print(data_emea.head())

print("\nSelected columns for DAO:")
print(data_dao.head())

print("\nSelected columns for APJ:")
print(data_apj.head())


Selected columns for EMEA:
             DateTime  ORDER_COUNT
0 2024-02-07 09:00:00          831
1 2024-02-07 10:00:00          870
2 2024-02-07 11:00:00          897
3 2024-02-07 12:00:00          875
4 2024-02-07 13:00:00          924

Selected columns for DAO:
             DateTime  ORDER_COUNT
0 2024-02-07 09:00:00           81
1 2024-02-07 10:00:00          111
2 2024-02-07 11:00:00          184
3 2024-02-07 12:00:00          729
4 2024-02-07 13:00:00          999

Selected columns for APJ:
             DateTime  ORDER_COUNT
0 2024-02-07 09:00:00          508
1 2024-02-07 10:00:00          377
2 2024-02-07 11:00:00          264
3 2024-02-07 12:00:00          146
4 2024-02-07 13:00:00           97


In [38]:
# Forecasting Models for 3 Regions
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

def process_region(data, region_name):
    # Feature engineering for seasonality
    data['hour'] = data.index.hour
    data['dayofweek'] = data.index.dayofweek
    data['month'] = data.index.month

    # Prepare features and target with seasonality
    X = data[['hour', 'dayofweek', 'month']]  # Add more features as needed
    y = data['ORDER_COUNT']

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train your models here (Decision Tree, Random Forest, Gradient Boosting, SVM)
    # Decision Tree Regressor
    dt_model = DecisionTreeRegressor(random_state=42)
    dt_model.fit(X_train, y_train)
    dt_predictions = dt_model.predict(X_test)
    dt_mse = mean_squared_error(y_test, dt_predictions)
    print(f'{region_name} - Decision Tree MSE: {dt_mse}')

    # Random Forest Regressor
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    rf_predictions = rf_model.predict(X_test)
    rf_mse = mean_squared_error(y_test, rf_predictions)
    print(f'{region_name} - Random Forest MSE: {rf_mse}')

    # Gradient Boosting Regressor
    gbm_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    gbm_model.fit(X_train, y_train)
    gbm_predictions = gbm_model.predict(X_test)
    gbm_mse = mean_squared_error(y_test, gbm_predictions)
    print(f'{region_name} - GBM MSE: {gbm_mse}')

    # Support Vector Machine Regressor
    svm_model = SVR(kernel='rbf')
    svm_model.fit(X_train, y_train)
    svm_predictions = svm_model.predict(X_test)
    svm_mse = mean_squared_error(y_test, svm_predictions)
    print(f'{region_name} - SVM MSE: {svm_mse}')

    # Combine actual and forecasted values into DataFrames for comparison
    actual_vs_forecasted_dt = pd.DataFrame({'Actual': y_test, 'Forecasted_DT': dt_predictions})
    actual_vs_forecasted_rf = pd.DataFrame({'Actual': y_test, 'Forecasted_RF': rf_predictions})
    actual_vs_forecasted_gbm = pd.DataFrame({'Actual': y_test, 'Forecasted_GBM': gbm_predictions})
    actual_vs_forecasted_svm = pd.DataFrame({'Actual': y_test, 'Forecasted_SVM': svm_predictions})

    # Extend the datetime index for the next 100 future values with 1-hour interval
    last_timestamp = data.index[-1]
    future_dates = pd.date_range(start=last_timestamp, periods=167, freq='H')[1:]

    # Create a DataFrame for future dates and engineer seasonality features
    future_df = pd.DataFrame(index=future_dates)
    future_df['hour'] = future_df.index.hour
    future_df['dayofweek'] = future_df.index.dayofweek
    future_df['month'] = future_df.index.month

    # Predict the next 100 future values using each model
    future_predictions_dt = dt_model.predict(future_df)
    future_predictions_rf = rf_model.predict(future_df)
    future_predictions_gbm = gbm_model.predict(future_df)
    future_predictions_svm = svm_model.predict(future_df)

    # Create a DataFrame for the future predictions
    future_predictions_df = pd.DataFrame({
        'DateTime': future_dates,
        'Forecasted_DT': future_predictions_dt,
        'Forecasted_RF': future_predictions_rf,
        'Forecasted_GBM': future_predictions_gbm,
        'Forecasted_SVM': future_predictions_svm
    })

    # Set the DateTime as the index
    future_predictions_df.set_index('DateTime', inplace=True)

    # Combine actual vs forecasted and future forecasted values into a single DataFrame
    combined_results_df = pd.concat([
        actual_vs_forecasted_dt,
        actual_vs_forecasted_rf[['Forecasted_RF']],
        actual_vs_forecasted_gbm[['Forecasted_GBM']],
        actual_vs_forecasted_svm[['Forecasted_SVM']],
        future_predictions_df
    ], axis=1)

    # Define the output path for the Excel file
    output_excel_path = f'ML_forecast_results_{region_name}_.xlsx'

    # Export the DataFrame to an Excel file
    combined_results_df.to_excel(output_excel_path, index=True)

    print(f"Data exported to {output_excel_path} for {region_name}")

# Load your datasets
data_emea = pd.DataFrame(data_emea)
data_emea['DateTime'] = pd.to_datetime(data_emea['DateTime'])
data_emea.set_index('DateTime', inplace=True)
data_dao = pd.DataFrame(data_dao)
data_dao['DateTime'] = pd.to_datetime(data_dao['DateTime'])
data_dao.set_index('DateTime', inplace=True)
data_apj = pd.DataFrame(data_apj)
data_apj['DateTime'] = pd.to_datetime(data_apj['DateTime'])
data_apj.set_index('DateTime', inplace=True)


# Process each region
process_region(data_emea, 'EMEA')
process_region(data_dao, 'DAO')
process_region(data_apj, 'APJ')


EMEA - Decision Tree MSE: 25316.587668137876
EMEA - Random Forest MSE: 23184.29621400312
EMEA - GBM MSE: 16718.94683008091
EMEA - SVM MSE: 96897.07505086665


C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_43120\316647103.py:60: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(start=last_timestamp, periods=167, freq='H')[1:]


Data exported to ML_forecast_results_EMEA_.xlsx for EMEA
DAO - Decision Tree MSE: 283180.62128433783
DAO - Random Forest MSE: 262584.7768208985
DAO - GBM MSE: 202656.01606162518
DAO - SVM MSE: 1244551.7605709522


C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_43120\316647103.py:60: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(start=last_timestamp, periods=167, freq='H')[1:]


Data exported to ML_forecast_results_DAO_.xlsx for DAO
APJ - Decision Tree MSE: 35382.870161514904
APJ - Random Forest MSE: 32404.18142475029
APJ - GBM MSE: 23508.216303361118
APJ - SVM MSE: 81396.33703120888


C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_43120\316647103.py:60: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(start=last_timestamp, periods=167, freq='H')[1:]


Data exported to ML_forecast_results_APJ_.xlsx for APJ
